# Drón adatok feldolgozása

Ebben a háziban az University of Zürich (UZH) [Robotics and Perception Group](http://rpg.ifi.uzh.ch/) labor adatait kell feldolgoznotok. Ők önvezető drónokkal (is) foglalkoznak, mi pedig az egyik mérésük adatait fogjuk felhasználni, hogy néhány görbére jellemző paramétert meghatározzunk. Ehhez először az adatokat kell letölteni, amit az ez alatti cella futtatásával tudtok megtenni. Ha saját gépen dolgoztok, akkor lehetséges, hogy nem találja majd a rendszer a git parancsot, ekkor kézzel is le tudjátok tölteni a Github-ról. (A notebook mappájába csomagoljátok ki a letöltött .zip-et ilyenkor.)

In [1]:
! git clone https://github.com/uzh-rpg/mh_autotune/

fatal: destination path 'mh_autotune' already exists and is not an empty directory.


A háziban felhasznált csogat is telepíthetjük könnyen:

In [ ]:
! pip install plotly
! pip install pandas

Ha megvannak az adataink a mappánkban, akkor be is olvashatjuk a számunka érdekes adatokat abból. Erre a pandas csomagot fogjuk használni és egy dataframe-be rakjuk a dolgokat. Később ezeket az adatokat könnyen átalakíthatjuk a tetszőleges formában, de használhajuk a pandas beépített függvényeit is. 

In [6]:
import pandas as pd

df = pd.read_csv("mh_autotune/resources/trajectories/spiral.csv")
# Akik zip-ből csinálják:
# df = pd.read_csv("mh_autotune-main/mh_autotune-main/resources/trajectories/spiral.csv")
df 

,Unnamed: 0,t,p_x,p_y,p_z,q_w,q_x,q_y,q_z,v_x,v_y,v_z,w_x,w_y,w_z,u_1,u_2,u_3,u_4
0,0,0.000000,0.750000,2.000000,-2.020000,0.707107,-7.536130e-26,3.320919e-25,-0.707107,0.000000,0.000000,0.000000,3.627782,1.595086,-1.019633e-09,3.999997,3.999997,3.999997,3.999996
1,1,0.008737,0.749994,1.999997,-2.019764,0.707001,1.613351e-02,-6.279024e-03,-0.707001,-0.002214,-0.000973,0.054073,3.627782,1.595086,-4.997181e-34,3.999996,3.999998,3.999997,3.999995
2,2,0.017475,0.749948,1.999977,-2.019056,0.706683,3.226219e-02,-1.255616e-02,-0.706683,-0.008846,-0.003889,0.107855,3.627782,1.595085,4.638624e-38,3.999996,3.999999,3.999997,3.999994
3,3,0.026212,0.749826,1.999924,-2.017880,0.706153,4.838119e-02,-1.882954e-02,-0.706153,-0.019877,-0.008740,0.161178,3.627780,1.595084,-9.149956e-34,3.999995,3.999999,3.999996,3.999992
4,4,0.034950,0.749588,1.999819,-2.016241,0.705412,6.448568e-02,-2.509727e-02,-0.705412,-0.035286,-0.015515,0.213878,3.627779,1.595083,-1.213455e-28,3.999994,3.999999,3.999996,3.999990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3776,3776,32.992700,-16.243308,-10.728720,3.965199,0.616491,-3.988703e-01,2.369371e-01,0.636162,-13.675455,17.902855,-0.533332,0.015265,0.015003,-1.933826e-28,3.999844,3.999844,3.999843,3.999843
3777,3777,33.001500,-16.362791,-10.571990,3.960519,0.616502,-3.988709e-01,2.370199e-01,0.636120,-13.674069,17.972460,-0.538124,0.015265,0.015003,1.433823e-34,3.999781,3.999781,3.999781,3.999780
3778,3778,33.010200,-16.482262,-10.414652,3.955796,0.616513,-3.988715e-01,2.371027e-01,0.636078,-13.672667,18.041917,-0.542917,0.015265,0.015002,6.342295e-30,3.999634,3.999634,3.999634,3.999634
3779,3779,33.019000,-16.601721,-10.256708,3.951031,0.616524,-3.988720e-01,2.371855e-01,0.636036,-13.671248,18.111224,-0.547713,0.015265,0.015002,7.016110e-30,3.998898,3.998898,3.998898,3.998898


Ebben a formában lehetetlen megmondani, hogy milyen pályát írt le a drón a tesztpályán; érdemes az adatokat megjeleníteni. Erre legegyszerűbb a plotly-t használni, ahol könnyen készthetünk interaktív 3D ábrákat néhány sor kóddal. (Sajnos a plotly néha megvicceli az embert, ilyenkor érdemes újra tölteni a böngésző ablakát, hogy a webGL dolgokat rendberakja.)

In [7]:
# Notebook miatt kellhet
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Plotly beolvasása
import plotly.express as px

# Az egyes tengelyek a dataframe egyes oszlopai
fig = px.line_3d(df, x='p_x', y='p_y', z='p_z')
fig.show()

Így már könnyeben látszik, hogy egy spirállal van dolgunk. Az is jól látszik, hogy alig van zaja a mérésüknek, otthoni körülmények között azért ne ilyen adatokra számítsünk. Mivel ilyen kicsi a zaja, ezért a későbbiekben a deriváltakkal sem lesz nagy problémánk.

## Sebességek számítása

A középiskolai fizikát felhasználva egyszerűen számíthatjuk a sebességeket a megtett távolság és az eközben eltelt idő alapján. Ehhez a számítháshoz használhajuk a pandas shift függvényét, amivel az adatainkat tudjuk a függvényben megadott létésben eltöltni. A sebesség nagyságát ezek után numpy segítségével számítjuk és létrehozunk egy új oszlopot, hogy eltároljuk. 

In [8]:
import numpy as np

df["p_x_d"] = (df["p_x"]-df["p_x"].shift(1))/(df["t"]-df["t"].shift(1))
df["p_y_d"] = (df["p_y"]-df["p_y"].shift(1))/(df["t"]-df["t"].shift(1))
df["p_z_d"] = (df["p_z"]-df["p_z"].shift(1))/(df["t"]-df["t"].shift(1))
df["p_v_abs"] = np.sqrt(df["p_z_d"].values**2+df["p_z_d"].values**2+df["p_z_d"].values**2)
df

,Unnamed: 0,t,p_x,p_y,p_z,q_w,q_x,q_y,q_z,v_x,...,w_y,w_z,u_1,u_2,u_3,u_4,p_x_d,p_y_d,p_z_d,p_v_abs
0,0,0.000000,0.750000,2.000000,-2.020000,0.707107,-7.536130e-26,3.320919e-25,-0.707107,0.000000,...,1.595086,-1.019633e-09,3.999997,3.999997,3.999997,3.999996,NaN,NaN,NaN,NaN
1,1,0.008737,0.749994,1.999997,-2.019764,0.707001,1.613351e-02,-6.279024e-03,-0.707001,-0.002214,...,1.595086,-4.997181e-34,3.999996,3.999998,3.999997,3.999995,-0.000738,-0.000325,0.027054,0.046859
2,2,0.017475,0.749948,1.999977,-2.019056,0.706683,3.226219e-02,-1.255616e-02,-0.706683,-0.008846,...,1.595085,4.638624e-38,3.999996,3.999999,3.999997,3.999994,-0.005162,-0.002270,0.080995,0.140288
3,3,0.026212,0.749826,1.999924,-2.017880,0.706153,4.838119e-02,-1.882954e-02,-0.706153,-0.019877,...,1.595084,-9.149956e-34,3.999995,3.999999,3.999996,3.999992,-0.013996,-0.006154,0.134562,0.233067
4,4,0.034950,0.749588,1.999819,-2.016241,0.705412,6.448568e-02,-2.509727e-02,-0.705412,-0.035286,...,1.595083,-1.213455e-28,3.999994,3.999999,3.999996,3.999990,-0.027218,-0.011967,0.187589,0.324913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3776,3776,32.992700,-16.243308,-10.728720,3.965199,0.616491,-3.988703e-01,2.369371e-01,0.636162,-13.675455,...,0.015003,-1.933826e-28,3.999844,3.999844,3.999843,3.999843,-13.735068,17.944980,-0.533225,0.923572
3777,3777,33.001500,-16.362791,-10.571990,3.960519,0.616502,-3.988709e-01,2.370199e-01,0.636120,-13.674069,...,0.015003,1.433823e-34,3.999781,3.999781,3.999781,3.999780,-13.577621,17.810244,-0.531923,0.921317
3778,3778,33.010200,-16.482262,-10.414652,3.955796,0.616513,-3.988715e-01,2.371027e-01,0.636078,-13.672667,...,0.015002,6.342295e-30,3.999634,3.999634,3.999634,3.999634,-13.732286,18.084791,-0.542850,0.940243
3779,3779,33.019000,-16.601721,-10.256708,3.951031,0.616524,-3.988720e-01,2.371855e-01,0.636036,-13.671248,...,0.015002,7.016110e-30,3.998898,3.998898,3.998898,3.998898,-13.574836,17.948171,-0.541442,0.937804


Ezek után meg is jelentíthetjük az abszolut segességet, mint színt a 3D-s ábránkon. 

In [9]:
fig = px.scatter_3d(df.iloc[1:,:], x = 'p_x', y = 'p_y', z = 'p_z', color = "p_v_abs")
fig.show()

A szemfülesek észrevehették, hogy a mérés tartalmaz sebesség adatokat is, nézzük meg, hogy mekkor az áltaunk számított és a mért közt a különbség.

In [13]:
fig = px.scatter(df.iloc[1:,:], x = 'p_x_d', y = 'v_x')
fig.show()

Nem tűnik olyan rossznak amit csináltunk, de ha belenagyítunk, akkor azért látszik, hogy van eltérés.

## Feladat 1. 

Határozzuk meg a gyorsulásokat, és jelenítsük meg az abszolút gyorsulást egy, a sebességet ábrázolóhoz hasonló 3D-s ábrán!

## Feladat 2. 

Határossuk meg a görbék torzióját és ábrázoljuk ezt is 3D ábán (a görbület skalárfüggvényét színként ábrázolva a görbén)!

## Feladat 3. 

Készítsünk egy függvényt, ami megadja a görbe adott pontjának kisérő triéder vektorait!


In [ ]:
def kisero_trieder(df = df, pont = 100):
    # dataframe-ben vannak megadott görbe pontjai 
    # a pont azt jelöli ki, hogy hanyadik görbe pont környezetében vagyunk kíváncsiak a kisérő triéderre. 
    #...
    return(t,n,b)